# Looking into the 3 CAVIAR "outliers"

Here we want to understand the examples that CAVIAR `-c 1` and susie `L 1` do not agree for `n = 1` causal variable.

The plan is to pin-point the data in question and get the corresponding data-set, then use interactive analysis to explore in detail.

## Extract simulated dataset

In [1]:
%cd ~/GIT/github/mvarbvs/dsc

/home/gaow/GIT/github/mvarbvs/dsc

In [5]:
dataset = c('~/Documents/GTExV8/Toys/Thyroid.ENSG00000144445.RDS', '~/Documents/GTExV8/Toys/Thyroid.ENSG00000155324.RDS', '~/Documents/GTExV8/Toys/Thyroid.ENSG00000156738.RDS')
out = dscrutils::dscquery('susie_comparison', 
                          targets = "liter_data.dataset lm_less.n_signal lm_less", 
                          conditions = "lm_less.n_signal = 1")

Loading dsc-query output from CSV file.


In [9]:
out[which(out$liter_data.dataset %in% dataset),]

,DSC,liter_data.dataset,lm_less.n_signal,lm_less.output.file
39,1,~/Documents/GTExV8/Toys/Thyroid.ENSG00000144445.RDS,1,lm_less/liter_data_39_summarize_ld_1_lm_less_1
48,1,~/Documents/GTExV8/Toys/Thyroid.ENSG00000155324.RDS,1,lm_less/liter_data_48_summarize_ld_1_lm_less_1
49,1,~/Documents/GTExV8/Toys/Thyroid.ENSG00000156738.RDS,1,lm_less/liter_data_49_summarize_ld_1_lm_less_1


In [11]:
bash:
    cp susie_comparison/lm_less/liter_data_{39,48,49}_summarize_ld_1_lm_less_1.pkl ../data